### **Data Transformation (or data augmentation) as per the AlexNet paper** ##

data augmentation (only image translation and horizontal reflection; read section 4.1 
on the paper)
- Random crop of 224x224 from the 256x256 image
- Random horizontal flipping

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import random

In [3]:
# Setting random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [4]:
import os

# Define dataset paths

dataset_root = 'alexnet_dataset'

# dataset_root = 'alexnet_dataset'
train_dir = os.path.join(dataset_root, 'train')
val_dir = os.path.join(dataset_root, 'val')
test_dir = os.path.join(dataset_root, 'test')

#### Why Class mapping? ####

In [5]:
# Load class mapping
class_to_idx = {}
with open(os.path.join(dataset_root, 'class_mapping.txt'), 'r') as f:
    for line in f:
        cls, idx = line.strip().split(',')
        class_to_idx[cls] = int(idx)

### Define Transformation for datasets ###

In [6]:
# Data transformation for training
train_transform = transforms.Compose([
    transforms.RandomCrop(224),  # Random crop to 224x224
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ToTensor(),  # Convert to tensor (scales to [0, 1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization

])

# Data transformation for validation/testing (no augmentation)
val_transform = transforms.Compose([
    transforms.CenterCrop(224),  # Center crop to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

In [7]:
# Custom Dataset class
class AlexNetDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        
        # Scan directory for images and labels
        class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        for class_name in class_dirs:
            class_idx = class_to_idx[class_name]
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(class_dir, img_name), class_idx))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        try:
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
                
            return image, label
        except Exception as e:
            print(f"Error loading image {idx}: {e}")
            # Return a placeholder instead of failing completely
            # This helps prevent worker crashes
            placeholder = torch.zeros((3, 224, 224)) if self.transform else Image.new('RGB', (224, 224))
            return placeholder, 0  # Return placeholder with dummy label

# Create datasets
train_dataset = AlexNetDataset(train_dir, transform=train_transform)
val_dataset = AlexNetDataset(val_dir, transform=val_transform)
test_dataset = AlexNetDataset(test_dir, transform=val_transform)

# Create data loaders
# AlexNet paper used a batch size of 128
batch_size = 128
# batch_size = 16
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,  
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False,
    # persistent_workers=False
    )
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0, 
    pin_memory=True if torch.cuda.is_available() else False,
    # persistent_workers=False
    )
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0, 
    pin_memory=True if torch.cuda.is_available() else False,
    persistent_workers=False
    )

print(f"Training set: {len(train_dataset)} images")
print(f"Validation set: {len(val_dataset)} images")
print(f"Testing set: {len(test_dataset)} images")
print(f"Number of classes: {len(class_to_idx)}")
print(f"Batch size: {batch_size}")
print("Data preparation complete and ready for AlexNet model training!")


# if __name__ == "__main__":
#     print(f"Training set: {len(train_dataset)} images")
#     print(f"Validation set: {len(val_dataset)} images")
#     print(f"Testing set: {len(test_dataset)} images")
#     print(f"Number of classes: {len(class_to_idx)}")
#     print(f"Batch size: {batch_size}")
#     print("Data preparation complete and ready for AlexNet model training!")


Training set: 30000 images
Validation set: 10000 images
Testing set: 10000 images
Number of classes: 100
Batch size: 128
Data preparation complete and ready for AlexNet model training!


In [8]:
# Add this right after creating your DataLoaders
print(f"Train loader num_workers: {train_loader.num_workers}")
print(f"Val loader num_workers: {val_loader.num_workers}")
# print(f"Test loader num_workers: {test_loader.num_workers}")

Train loader num_workers: 0
Val loader num_workers: 0


In [9]:
print(len(train_loader))

235


In [10]:
print(train_dataset[0][0])

tensor([[[ 0.1939,  0.3309,  0.3823,  ..., -0.2684, -0.3541, -0.4739],
         [-0.0458,  0.1083,  0.2453,  ..., -0.3541, -0.4397, -0.5253],
         [-0.3541, -0.1486,  0.0398,  ..., -0.4739, -0.4911, -0.5253],
         ...,
         [-1.0733, -0.9705, -0.8849,  ..., -0.6281, -0.6281, -0.6965],
         [-0.9705, -0.9363, -0.8849,  ..., -0.3712, -0.6452, -0.5938],
         [-1.0048, -0.9534, -0.9020,  ..., -0.1828, -0.4226, -0.3883]],

        [[ 0.2402,  0.3803,  0.3978,  ...,  0.1527,  0.1352,  0.1176],
         [ 0.1001,  0.2227,  0.2927,  ...,  0.1352,  0.1176,  0.1001],
         [-0.0749,  0.0301,  0.1527,  ...,  0.1176,  0.1527,  0.1702],
         ...,
         [-0.5826, -0.5301, -0.4426,  ..., -0.4251, -0.4776, -0.5301],
         [-0.5476, -0.5476, -0.4951,  ..., -0.2500, -0.5301, -0.5126],
         [-0.5826, -0.5651, -0.5126,  ..., -0.0574, -0.3025, -0.3025]],

        [[-0.4973, -0.3578, -0.3404,  ..., -0.8110, -0.8110, -0.8284],
         [-0.6715, -0.5321, -0.4450,  ..., -0